<a href="https://colab.research.google.com/github/gopinathak-geek/Image-classification-with-pytorch/blob/main/novo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/gopinathak-geek/novozymes-enzyme-stability-prediction.git

Cloning into 'novozymes-enzyme-stability-prediction'...
remote: Enumerating objects: 29516, done.
remote: Counting objects: 100% (143/143), done.
remote: Compressing objects: 100% (89/89), done.
remote: Total 29516 (delta 53), reused 141 (delta 52), pack-reused 29373
Receiving objects: 100% (29516/29516), 554.63 MiB | 17.09 MiB/s, done.
Resolving deltas: 100% (29188/29188), done.
Checking out files: 100% (29408/29408), done.


In [2]:
# Add this in a Google Colab cell to install the correct version of Pytorch Geometric.
import torch

def format_pytorch_version(version):
  return version.split('+')[0]

TORCH_version = torch.__version__
TORCH = format_pytorch_version(TORCH_version)

def format_cuda_version(version):
  return 'cu' + version.replace('.', '')

CUDA_version = torch.version.cuda
CUDA = format_cuda_version(CUDA_version)

!pip install torch-scatter     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-sparse      -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-cluster     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-geometric

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://pytorch-geometric.com/whl/torch-1.13.0+cu116.html
     |████████████████████████████████| 9.4 MB 22.4 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://pytorch-geometric.com/whl/torch-1.13.0+cu116.html
     |████████████████████████████████| 4.5 MB 32.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://pytorch-geometric.com/whl/torch-1.13.0+cu116.html
     |████████████████████████████████| 3.2 MB 39.6 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://pytorch-geometric.com/whl/torch-1.13.0+cu116.html
     |████████████████████████████████| 873 kB 21.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.d

In [3]:
!pip install rdkit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 29.3 MB 1.2 MB/s 


In [4]:
from rdkit import Chem
import torch
import torch_geometric
from torch_geometric.data import Data, Dataset, download_url, DataLoader
from torch_geometric.loader import DataLoader
import os.path as osp
import pandas as pd
import numpy as np
from tqdm import tqdm

In [5]:
class NovoDataset(Dataset):
    def __init__(self, root, filename, test=False, transform=None, pre_transform=None):
        self.test = test
        self.filename = filename
        super(NovoDataset, self).__init__(root, transform, pre_transform)

    @property
    def raw_file_names(self):
        return self.filename

    @property
    def processed_file_names(self):
        self.data = pd.read_csv(self.raw_paths[0]).reset_index()

        if self.test:
            return [f'data_test_{i}.pt' for i in list(self.data.index)]
        else:
            return [f'data_{i}.pt' for i in list(self.data.index)]

    def download(self):
        # Download to `self.raw_dir`.
        pass

    def process(self):
        self.data = pd.read_csv(self.raw_paths[0])
        for index, mol in tqdm(self.data.iterrows(), total=self.data.shape[0]):
            mol_obj = Chem.MolFromSequence(mol["protein_sequence"])
            ph = mol["pH"]
            
            # Get node features
            node_feats = self._get_node_features(mol_obj, ph)
            
            # Get edge features
            edge_feats = self._get_edge_features(mol_obj)
            
            # Get adjacency info
            edge_index = self._get_adjacency_info(mol_obj)
            
            
            if self.test:
                #Create data object for testing data
                data = Data(x=node_feats, edge_index=edge_index, edge_attr=edge_feats) 
                
                #save test data
                torch.save(data, osp.join(self.processed_dir, f'data_test_{index}.pt'))
            else:
                # Get labels info
                label = self._get_labels(mol["tm"])
                
                #Create data object for training data
                data = Data(x=node_feats, edge_index=edge_index, edge_attr=edge_feats, y=label) 
                
                #save train data
                torch.save(data, osp.join(self.processed_dir, f'data_{index}.pt'))
        
    def _get_node_features(self, mol, ph):
        """ 
        This will return a matrix / 2d array of the shape
        [Number of Nodes, Node Feature size]
        """
        all_node_feats = []
    
        for atom in mol.GetAtoms():
            node_feats = []
            # Feature 1: Atomic number        
            node_feats.append(atom.GetAtomicNum())
            # Feature 2: Atom degree
            node_feats.append(atom.GetDegree())
            # Feature 3: Formal charge
            node_feats.append(atom.GetFormalCharge())
            # Feature 4: Hybridization
            node_feats.append(atom.GetHybridization())
            # Feature 5: Aromaticity
            node_feats.append(atom.GetIsAromatic())
            # Feature 6: Total Num Hs
            node_feats.append(atom.GetTotalNumHs())
            # Feature 7: Radical Electrons
            node_feats.append(atom.GetNumRadicalElectrons())
            # Feature 8: In Ring
            node_feats.append(atom.IsInRing())
            # Feature 9: Chirality
            node_feats.append(atom.GetChiralTag())
            # Feature 10: ph
            node_feats.append(ph)
    
            # Append node features to matrix
            all_node_feats.append(node_feats)

        all_node_feats = np.asarray(all_node_feats)
        return torch.tensor(all_node_feats, dtype=torch.float)

    def _get_edge_features(self, mol):
        """ 
        This will return a matrix / 2d array of the shape
        [Number of edges, Edge Feature size]
        """
        all_edge_feats = []

        for bond in mol.GetBonds():
            edge_feats = []
            # Feature 1: Bond type (as double)
            edge_feats.append(bond.GetBondTypeAsDouble())
            # Feature 2: Rings
            edge_feats.append(bond.IsInRing())
            # Append node features to matrix (twice, per direction)
            all_edge_feats += [edge_feats, edge_feats]

        all_edge_feats = np.asarray(all_edge_feats)
        return torch.tensor(all_edge_feats, dtype=torch.float)

    def _get_adjacency_info(self, mol):
        """
        We could also use rdmolops.GetAdjacencyMatrix(mol)
        but we want to be sure that the order of the indices
        matches the order of the edge features
        """
        edge_indices = []
        for bond in mol.GetBonds():
            i = bond.GetBeginAtomIdx()
            j = bond.GetEndAtomIdx()
            edge_indices += [[i, j], [j, i]]

        edge_indices = torch.tensor(edge_indices)
        edge_indices = edge_indices.t().to(torch.long).view(2, -1)
        return edge_indices

    def _get_labels(self, label):
        return torch.tensor(label, dtype=torch.float)
    
    def len(self):
        return self.data.shape[0]

    def get(self, idx):
        """ - Equivalent to __getitem__ in pytorch
            - Is not needed for PyG's InMemoryDataset
        """
        if self.test:
            data = torch.load(osp.join(self.processed_dir, f'data_test_{idx}.pt'))
        else:
            data = torch.load(osp.join(self.processed_dir, f'data_{idx}.pt'))   
        return data

In [6]:
train_dataset = NovoDataset(root="novozymes-enzyme-stability-prediction/data/", filename="training.csv")
test_dataset = NovoDataset(root="novozymes-enzyme-stability-prediction/data/", filename="testing.csv", test=True)

In [7]:
import torch
from torch_geometric.loader import DataLoader
import torch.nn as nn
import torch.nn.functional as F 
from torch.nn import Linear, BatchNorm1d, ModuleList
from torch_geometric.nn import TransformerConv, TopKPooling , GCNConv, TopKPooling, global_mean_pool
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp

In [8]:
train_size = int(0.7 * len(train_dataset))
val_size = len(train_dataset) - train_size
training_data, validation_data = torch.utils.data.random_split(train_dataset, [train_size, val_size])

In [9]:
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
BATCHSIZE = 128
OUTPUT = 1
EMBEDDING_SIZE = 64

In [10]:
train_loader = DataLoader(dataset=training_data, batch_size=BATCHSIZE, shuffle=True, num_workers=2, pin_memory=True)
valid_loader = DataLoader(dataset=validation_data, batch_size=BATCHSIZE, shuffle=False, num_workers=2, pin_memory=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=1, shuffle=False, num_workers=2, pin_memory=True)

In [37]:
class GCN(torch.nn.Module):
    def __init__(self):
        # Init parent
        super(GCN, self).__init__()
        torch.manual_seed(42)

        # GCN layers
        self.initial_conv = GCNConv(10, EMBEDDING_SIZE)
        self.conv1 = GCNConv(EMBEDDING_SIZE, EMBEDDING_SIZE)
        self.conv2 = GCNConv(EMBEDDING_SIZE, EMBEDDING_SIZE)
        self.conv3 = GCNConv(EMBEDDING_SIZE, EMBEDDING_SIZE)

        # Output layer
        self.out = Linear(EMBEDDING_SIZE*2, 1)

    def forward(self, x, edge_index, batch_index):
        # First Conv layer
        hidden = self.initial_conv(x, edge_index)
        hidden = torch.tanh(hidden)

        # Other Conv layers
        hidden = self.conv1(hidden, edge_index)
        hidden = torch.tanh(hidden)
        #hidden = self.conv2(hidden, edge_index)
        #hidden = torch.tanh(hidden)
        #hidden = self.conv3(hidden, edge_index)
        #hidden = torch.tanh(hidden)
          
        # Global Pooling (stack different aggregations)
        hidden = torch.cat([gmp(hidden, batch_index), 
                            gap(hidden, batch_index)], dim=1)

        # Apply a final (linear) classifier.
        out = self.out(hidden)

        return out, hidden

In [38]:
model = GCN()
model.to(DEVICE)
criterion = nn.MSELoss()
LEARNING_RATE = 0.001
EPOCHS = 20
optimizer = torch.optim.RMSprop(model.parameters(), lr=LEARNING_RATE)

In [39]:
from scipy import stats


In [40]:
training_score_history = []
training_losses_history = []
validation_score_history = []
validation_losses_history = []
for epoch in range(EPOCHS):
    model.train()
    training_score = []
    training_loss = []
    for batch in train_loader:
        batch.to(DEVICE)
        label = batch.y[:, None] 

        #==========Forward pass===============
        pred, embedding = model(batch.x.float(), batch.edge_index, batch.batch)
        loss = criterion(pred, label)
        #==========backward pass==============
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        train_result = stats.spearmanr(pred.detach().cpu().numpy(), label.cpu().numpy())
        training_score.append(train_result.correlation)
        training_loss.append(loss.item())
    
    training_scores = np.mean(training_score)
    training_losses = np.mean(training_loss)

    validation_score = []
    validation_loss = []
    for batch in valid_loader:
        model.eval()
        with torch.no_grad():
            batch.to(DEVICE)
            label = batch.y[:, None] 
            val_preds, val_embedding = model(batch.x.float(), batch.edge_index, batch.batch)
            val_loss = criterion(val_preds, label)
            
            val_result = stats.spearmanr(val_preds.detach().cpu().numpy(), label.cpu().numpy())
            validation_score.append(val_result.correlation)
            validation_loss.append(val_loss.item())
        
    validation_scores = np.mean(validation_score)
    validation_losses = np.mean(validation_loss)
        
    training_score_history.append(training_scores)
    training_losses_history.append(training_losses)
    validation_score_history.append(validation_scores)
    validation_losses_history.append(validation_losses)
    print(f'{epoch+1:03} EPOCH - Training score : {np.mean(training_scores):.5f} | Validation score : {np.mean(validation_scores):.5f} | Training loss : {np.mean(training_losses):.5f} | Validation loss : {np.mean(validation_losses):.5f}')

001 EPOCH - Training score : -0.07438 | Validation score : -0.02316 | Training loss : 1012.67951 | Validation loss : 463.96395
002 EPOCH - Training score : -0.03726 | Validation score : -0.00147 | Training loss : 288.71066 | Validation loss : 184.57563
003 EPOCH - Training score : -0.02330 | Validation score : 0.01069 | Training loss : 155.64786 | Validation loss : 147.43455
004 EPOCH - Training score : -0.01069 | Validation score : 0.01984 | Training loss : 143.77202 | Validation loss : 146.53795
005 EPOCH - Training score : -0.00594 | Validation score : 0.02242 | Training loss : 143.91967 | Validation loss : 146.51033
006 EPOCH - Training score : -0.00669 | Validation score : 0.02355 | Training loss : 143.79747 | Validation loss : 146.58112
007 EPOCH - Training score : -0.00245 | Validation score : 0.02721 | Training loss : 143.69523 | Validation loss : 146.57045
008 EPOCH - Training score : 0.00095 | Validation score : 0.02953 | Training loss : 143.73758 | Validation loss : 146.5227

In [41]:
def predections(dataloader):
  test_tm_predictions = []
  model.eval()
  with torch.no_grad():
    for batch in dataloader:
      batch.to(DEVICE)
      tm_prediction, tm_embedding = model(batch.x.float(), batch.edge_index, batch.batch)
      test_tm_predictions.append(tm_prediction.cpu().numpy())
  return [tm.squeeze().tolist() for tm in test_tm_predictions]

In [42]:
tm = predections(test_loader)

In [43]:
test_csv = "https://raw.githubusercontent.com/gopinathak-geek/novozymes-enzyme-stability-prediction/main/data/raw/testing.csv"
test_df = pd.read_csv(test_csv)
test_seq_id = test_df['seq_id']

In [44]:
df = pd.DataFrame({"seq_id":test_seq_id, "tm":tm })
df

,seq_id,tm
0,31390,52.541363
1,31391,52.541370
2,31392,52.540695
3,31393,52.541069
4,31394,52.536831
...,...,...
2408,33798,52.541515
2409,33799,52.541515
2410,33800,52.541515
2411,33801,52.541431


In [45]:
df.describe()[['tm']].transpose()


,count,mean,std,min,25%,50%,75%,max
tm,2413.0,52.540711,0.002771,52.532303,52.540997,52.541229,52.541443,52.550316


In [46]:
df.to_csv("submission.csv", index=False)